<a href="https://colab.research.google.com/github/LeeGitaek/2020_AI_Class/blob/master/softmax_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Softmax classification

클래스가 3개 이상인 경우

데이터 로드


In [18]:
import torch
import torch.optim as optim
# For reproducibility
torch.manual_seed(1)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]

y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

print(x_train[:5])
print(y_train[:5])


tensor([[1., 2., 1., 1.],
        [2., 1., 3., 2.],
        [3., 1., 3., 4.],
        [4., 1., 5., 5.],
        [1., 7., 5., 5.]])
tensor([2, 2, 2, 1, 1])


In [19]:
print(y_train)
print(y_train.unsqueeze(1))

nb_class = 3
nb_data = len(y_train)
y_one_hot = torch.zeros(nb_data, nb_class)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

print(y_one_hot)


tensor([2, 2, 2, 1, 1, 1, 0, 0])
tensor([[2],
        [2],
        [2],
        [1],
        [1],
        [1],
        [0],
        [0]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


모델 학습


In [20]:
import torch.nn.functional as F # for softmax

nb_class = 3
nb_data = len(y_train)

w = torch.zeros((4, nb_class),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([w,b], lr = 0.01)
nb_epochs = 1000
for epoch in range(nb_epochs+1):

    hypothesis = F.softmax(x_train.matmul(w)+b, dim=1)

    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    #cost 표현법 2번 예시 (위 1번 ) 
    # cross_ entropy 를 사용하면 scatter 함수를 이용한 one_hot_encoding을 안해도됨.
    # cost = F.cross_entropy((x_train.matmul(w)+b),y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('epoch {:4d}/{} cost : {:.6f}'.format(
            epoch, nb_epochs,cost.item()
        ))
    


epoch    0/1000 cost : 1.098612
epoch  100/1000 cost : 1.036919
epoch  200/1000 cost : 1.010905
epoch  300/1000 cost : 0.991541
epoch  400/1000 cost : 0.974942
epoch  500/1000 cost : 0.959928
epoch  600/1000 cost : 0.946149
epoch  700/1000 cost : 0.933492
epoch  800/1000 cost : 0.921898
epoch  900/1000 cost : 0.911302
epoch 1000/1000 cost : 0.901625


데이터 평가 



In [26]:
#학습된 w,b 를 통한 클래스 예측 
hypothesis = F.softmax(x_train.matmul(w) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)


print(hypothesis)
print(predict)
print(y_train)


# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))


tensor([[0.3633, 0.1321, 0.5046],
        [0.1075, 0.3747, 0.5178],
        [0.0338, 0.4811, 0.4851],
        [0.0120, 0.5703, 0.4177],
        [0.9002, 0.0161, 0.0838],
        [0.1476, 0.7789, 0.0735],
        [0.8375, 0.0747, 0.0878],
        [0.8986, 0.0490, 0.0524]], grad_fn=<SoftmaxBackward>)
tensor([2, 2, 2, 1, 0, 1, 0, 0])
tensor([2, 2, 2, 1, 1, 1, 0, 0])
tensor([ True,  True,  True,  True, False,  True,  True,  True])
The model has an accuracy of 87.50% for the training set.
